In [6]:
import ee
import geemap
import streamlit as st
import streamlit.components.v1 as components

# Initialize Earth Engine
ee.Initialize()

# Define the area of interest (Prague)
prague = ee.Geometry.Point(14.4378, 50.0755).buffer(20000)

# Define the time range
start_date = '2020-7-01'
end_date = '2021-12-31'

# Select the Sentinel-2 Surface Reflectance dataset
collection = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(prague) \
    .filterDate(start_date, end_date)

# Function to create monthly composites
def create_monthly_composite(year, month):
    start = ee.Date.fromYMD(year, month, 1)
    end = start.advance(1, 'month')
    monthly = collection.filterDate(start, end).median()
    return monthly.set('year', year).set('month', month)

# Create a list of all month/year combinations
dates = [
    (year, month)
    for year in range(2020, 2022)
    for month in range(1, 13)
]

# Create monthly composites
monthly_composites = ee.ImageCollection(
    ee.List([create_monthly_composite(year, month) for year, month in dates])
)

# Function to export an image
def export_image(image, filename):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=filename,
        scale=10,
        region=prague,
        maxPixels=1e13
    )
    task.start()
    return task

# Export all monthly composites
tasks = []
for year, month in dates:
        image = monthly_composites.filter(ee.Filter.And(
            ee.Filter.eq('year', year),
            ee.Filter.eq('month', month)
        )).first()
        filename = f'Prague_Composite_{year}_{month:02d}'
        task = export_image(image, filename)
        tasks.append(task)

# Monitor task status
for task in tasks:
    print(f"Task {task.status()['description']}: {task.status()['state']}")


/home/basta/.virtualenvs/aiaccesibility/lib/python3.10/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2_SR! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

  warnings.warn(warning, category=DeprecationWarning)


Task Prague_Composite_2020_01: READY
Task Prague_Composite_2020_02: READY
Task Prague_Composite_2020_03: READY
Task Prague_Composite_2020_04: READY
Task Prague_Composite_2020_05: READY
Task Prague_Composite_2020_06: READY
Task Prague_Composite_2020_07: READY
Task Prague_Composite_2020_08: READY
Task Prague_Composite_2020_09: READY
Task Prague_Composite_2020_10: READY
Task Prague_Composite_2020_11: READY
Task Prague_Composite_2020_12: READY
Task Prague_Composite_2021_01: READY
Task Prague_Composite_2021_02: READY
Task Prague_Composite_2021_03: READY
Task Prague_Composite_2021_04: READY
Task Prague_Composite_2021_05: READY
Task Prague_Composite_2021_06: READY
Task Prague_Composite_2021_07: READY
Task Prague_Composite_2021_08: READY
Task Prague_Composite_2021_09: READY
Task Prague_Composite_2021_10: READY
Task Prague_Composite_2021_11: READY
Task Prague_Composite_2021_12: READY


In [7]:
monthly_composites